In [1]:
from distribute_zarr_overall_nodes import *
from utils.write_tools import node_assignment

time_step = 1

In [2]:
def transform_idxs_to_job(
    xs_ys_zs:Tuple[GivernyIndex, GivernyIndex, GivernyIndex]
) -> Tuple[str, Tuple[GivernyIndex, GivernyIndex, GivernyIndex]]:
    x, y, z = convert_idx(xs_ys_zs)
    node_assignment = assignable_nodes[node_assignments[x, y, z]]
    return (time_step, folder_name_idxs, node_assignment, xs_ys_zs)

In [3]:
assignable_nodes = list(map(
    lambda xy: "data{:02}_{:02}".format(*xy),
    filterfalse(
        lambda xy: (xy[0]==9 and xy[1]==2) or (xy[0]==7 and xy[1]==2),
        product(range(1, 13), range(1, 4)),
    )
))

In [4]:
x_ranges = get_range(1,10)
y_ranges = get_range(1,10)
z_ranges = get_range(1,10)

idxs = list(product(x_ranges, y_ranges, z_ranges))

In [5]:
node_assignments = node_assignment(cube_side=4) - 1
# node_assignments

In [6]:
folder_name_idxs = sorted(
    assignable_nodes, 
    key=lambda s: (s[-2:], s[4:6])
)

In [7]:
assignments_idxs = list(map(transform_idxs_to_job, idxs))
# assignments_idxs

In [8]:
idxs[0]

((-501, 10), (-501, 10), (-501, 10))

In [11]:
a = retrieve_data(idxs[0])

Note: For larger boxes, e.g. 512-cubed and up, processing will take approximately 1 minute or more...
-----

Step 1: Determining which database files the user-specified box is found in...
-------------------------
number of database files that the user-specified box is found in:
8

number of hard disks that the database files are distributed on:
5

Successfully completed.
-----

Step 2: Reading the data from all of the database files and storing the values into a matrix...
-------------------------
Database files are being read in parallel...

Successfully completed.
-----

Step 3: Writing the output matrix to a hdf5 file...
-------------------------

Successfully completed.
-----

step 1 time elapsed = 0.025 seconds (0.000 minutes)
step 2 time elapsed = 12.949 seconds (0.216 minutes)
step 3 time elapsed = 0.000 seconds (0.000 minutes)
total time elapsed = 13.008 seconds (0.217 minutes)

Data processing pipeline has completed successfully.
-----


In [12]:
a

<xarray.Dataset>
Dimensions:        (zcoor: 512, ycoor: 512, xcoor: 512, values: 1)
Coordinates:
  * zcoor          (zcoor) float32 -0.385 -0.3843 -0.3835 ... 0.006136 0.006903
  * ycoor          (ycoor) float32 -0.385 -0.3843 -0.3835 ... 0.006136 0.006903
  * xcoor          (xcoor) float32 -0.385 -0.3843 -0.3835 ... 0.006136 0.006903
Dimensions without coordinates: values
Data variables:
    Pressure_0001  (zcoor, ycoor, xcoor, values) float32 -3.005 ... -2.145
Attributes: (12/14)
    dataset:      isotropic8192
    t_start:      1
    t_end:        1
    t_step:       1
    x_start:      -501
    y_start:      -501
    ...           ...
    y_end:        10
    z_end:        10
    x_step:       1
    y_step:       1
    z_step:       1
    filterWidth:  1

In [13]:
a.info

<bound method Dataset.info of <xarray.Dataset>
Dimensions:        (zcoor: 512, ycoor: 512, xcoor: 512, values: 1)
Coordinates:
  * zcoor          (zcoor) float32 -0.385 -0.3843 -0.3835 ... 0.006136 0.006903
  * ycoor          (ycoor) float32 -0.385 -0.3843 -0.3835 ... 0.006136 0.006903
  * xcoor          (xcoor) float32 -0.385 -0.3843 -0.3835 ... 0.006136 0.006903
Dimensions without coordinates: values
Data variables:
    Pressure_0001  (zcoor, ycoor, xcoor, values) float32 -3.005 ... -2.145
Attributes: (12/14)
    dataset:      isotropic8192
    t_start:      1
    t_end:        1
    t_step:       1
    x_start:      -501
    y_start:      -501
    ...           ...
    y_end:        10
    z_end:        10
    x_step:       1
    y_step:       1
    z_step:       1
    filterWidth:  1>

## Unfortunately Ryan's code just converts binary to Zarr. The dataset is already distributed across FileDB and it's already in (512,512,512,1)